### Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("max_columns",500)
pd.set_option("max_rows",500)
import pickle
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

### Loading the dataset

In [2]:
movies_df=pd.read_csv("./data/tmdb_5000_movies.csv")
credits_df=pd.read_csv("./data/tmdb_5000_credits.csv")

In [3]:
print(f'Rows : {movies_df.shape[0]} Columns: {movies_df.shape[1]}')

movies_df.columns

Rows : 4803 Columns: 20


Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
print(f'Rows : {credits_df.shape[0]} Columns: {credits_df.shape[1]}')

credits_df.columns

Rows : 4803 Columns: 4


Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

### Merging both the dataframes on "title"

In [5]:
movies_df=pd.merge(left=movies_df,right=credits_df,how='inner',on='title')
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


### Columns to be Kept
> genres

> id

> keywords

> title

> overview

> cast 

> crew



### Filtering the dataset with the desired columns

In [6]:
movies=movies_df[['movie_id','title','overview','genres','keywords','cast','crew']]

In [7]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


### Now we want our dataframe to be like this
- movie_id
- title
- tags   --> we will merge [ overview,genres,keywords,cast,crew]

### Data Preprocessing

> Checking for null values

In [8]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

- overview has 3 missing values
- since it is a very important column for the recommendation engine we will not try to impute it by any means
- dropping it will be a right move

> Dropping null values

In [9]:
movies.dropna(inplace=True)

In [10]:
movies.duplicated().sum()

0

> No duplicated values in the dataset

In [11]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


> ### Data Cleaning

#### Cleaning Genres Column

In [12]:
def fetchGenreNames(obj):
    return [dicts['name'] for dicts in literal_eval(obj)]

movies['genres']=movies['genres'].apply(fetchGenreNames)

#### Cleaning keyword column

In [13]:
movies['keywords']=movies['keywords'].apply(fetchGenreNames)

#### Cleaning cast column
- fetching only top 3 cast from the movie

In [14]:
def getTop3Cast(obj):
    actor_names=[]
    counter=0
    for i in literal_eval(obj):
        if counter !=3:
            actor_names.append(i['name'])
            counter+=1
        else:
            break
    return actor_names
        
movies['cast']=movies['cast'].apply(getTop3Cast)

#### Cleaning crew column
- fetching only the directors name

In [15]:
def fetchDirectorName(obj):
    producerName=[]
    for i in literal_eval(obj):
        if i['job']=='Director':
            producerName.append(i['name'])
            break
    return producerName

movies['crew']=movies['crew'].apply(fetchDirectorName)

#### Converting the overview column into a list

In [16]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

##### Now the next step is to combine all these columns [overview genres keyword cast crew] to create tags column
- but before this we need to remove spaces 

    - for example 'Sam Worthington' to this 'SamWorthington'
    - so that 'Sam' and 'Worthington' will not be treated as a seperate entity

In [17]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [18]:
# removing spaces from genres, keywords, cast, crew

movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])

### Creating tags column

In [19]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

#### Creating new dataframe with movie_id , title and tags column

In [20]:
new_df=movies[['movie_id','title','tags']]
new_df

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


#### Converting tags column into a string

In [21]:
new_df['tags']=new_df['tags'].apply(lambda x : ' '.join(x))

In [22]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

#### Converting tags into lower case

In [23]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [24]:
new_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


### Applying Stemming on the tags
- this will treat similar words ***[loved , loving , love ]*** as ***love*** and other similar words like that

In [25]:
### creating an object of porter stemmer
ps=PorterStemmer()

In [26]:
def stem(text):
    stemmed_words=[]

    for i in text.split():
        stemmed_words.append(ps.stem(i))
    return ' '.join(stemmed_words)

new_df['tags']=new_df['tags'].apply(stem)

### Now we need to recommend movies based on the closest matching tags, but tags in in textual form
- #### so for this we will convert tags into vectors using ***Bag of Words*** techniuque
- #### and in this we will not consider **Stop Words**

> Stop Words are those words ...which are used in sentence formation but they do not contribute in the meaning of the sentence for example "[and] [are] [if] [to] [from]"

### Creating an object of CountVectorizer 

In [27]:
cv=CountVectorizer(max_features=5000,stop_words='english')

"""
By default this function will return sparse matrix so we are explicitly converting this into numpy array
"""

'\nBy default this function will return sparse matrix so we are explicitly converting this into numpy array\n'

### Transforming the tags into vectors

In [28]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [29]:
cv.get_feature_names()[100:110]

['adrienbrodi',
 'adult',
 'adultanim',
 'adulteri',
 'adulthood',
 'advanc',
 'adventur',
 'adventure',
 'adventures',
 'advertis']

### Calculating distances of each movie with all movies

In [30]:
similarity=cosine_similarity(vectors)

#### Here is the similarity score of each movie with all the movies
- and the similarity of a movie with itself will be 1 that is why in diagonal the value will always be 1

In [31]:
pd.DataFrame(similarity)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,4556,4557,4558,4559,4560,4561,4562,4563,4564,4565,4566,4567,4568,4569,4570,4571,4572,4573,4574,4575,4576,4577,4578,4579,4580,4581,4582,4583,4584,4585,4586,4587,4588,4589,4590,4591,4592,4593,4594,4595,4596,4597,4598,4599,4600,4601,4602,4603,4604,4605,4606,4607,4608,4609,4610,4611,4612,4613,4614,4615,4616,4617,4618,4619,4620,4621,4622,4623,4624,4625,4626,4627,4628,4629,4630,4631,4632,4633,4634,4635,4636,4637,4638,4639,4640,4641,4642,4643,4644,4645,4646,4647,4648,4649,4650,4651,4652,4653,4654,4655,4656,4657,4658,4659,4660,4661,4662,4663,4664,4665,4666,4667,4668,4669,4670,4671,4672,4673,4674,4675,4676,4677,4678,4679,4680,4681,4682,4683,4684,4685,4686,4687,4688,4689,4690,4691,4692,4693,4694,4695,4696,4697,4698,4699,4700,4701,4702,4703,4704,4705,4706,4707,4708,4709,4710,4711,4712,4713,4714,4715,4716,4717,4718,4719,4720,4721,4722,4723,4724,4725,4726,4727,4728,4729,4730,4731,4732,4733,4734,4735,4736,4737,4738,4739,4740,4741,4742,4743,4744,4745,4746,4747,4748,4749,4750,4751,4752,4753,4754,4755,4756,4757,4758,4759,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802,4803,4804,4805
0,1.000000,0.083462,0.086031,0.073472,0.189299,0.108389,0.040242,0.146735,0.059235,0.096730,0.102598,0.094650,0.090371,0.044992,0.128247,0.062828,0.078947,0.139777,0.094933,0.083081,0.058038,0.109682,0.066227,0.087407,0.053338,0.051016,0.153897,0.186933,0.116543,0.065033,0.066848,0.159071,0.085203,0.097333,0.000000,0.099340,0.173170,0.078947,0.081111,0.082261,0.076948,0.165637,0.000000,0.090862,0.033826,0.082409,0.139104,0.196722,0.084478,0.058272,0.112956,0.080484,0.148431,0.042974,0.031220,0.070382,0.138343,0.024183,0.067488,0.072548,0.018368,0.231793,0.083771,0.081752,0.029617,0.019529,0.022496,0.164523,0.120495,0.026491,0.076004,0.073367,0.137102,0.033113,0.202465,0.103905,0.071755,0.014395,0.050669,0.062311,0.050465,0.110208,0.076004,0.179969,0.123613,0.114127,0.070987,0.111283,0.090371,0.057579,0.061314,0.166924,0.062311,0.070987,0.187317,0.140488,0.076948,0.069523,0.075094,0.026669,0.067651,0.076948,0.124622,0.079872,0.055641,0.103757,0.062311,0.061806,0.145095,0.064889,0.044992,0.150188,0.076948,0.043355,0.045883,0.043748,0.048365,0.059235,0.033643,0.043355,0.000000,0.111283,0.116543,0.112480,0.067488,0.078433,0.133345,0.146011,0.000000,0.092698,0.027037,0.105263,0.044992,0.028239,0.129777,0.000000,0.023415,0.030124,0.128247,0.120727,0.044992,0.087407,0.084717,0.086711,0.050063,0.019252,0.016304,0.055641,0.097333,0.155113,0.164262,0.202953,0.083462,0.078689,0.061314,0.020438,0.059904,0.099340,0.133697,0.063628,0.117123,0.031814,0.033113,0.050669,0.080007,0.045883,0.125157,0.090371,0.175219,0.088302,0.106504,0.022716,0.212398,0.023174,0.077929,0.070987,0.029136,0.000000,0.024456,0.072548,0.022076,0.044992,0.117073,0.070382,0.177343,0.050669,0.059017,0.153897,0.022076,0.114708,0.084478,0.048912,0.047325,0.078782,0.000000,0.059235,0.051952,0.036974,0.057354,0.086711,0.108821,0.000000,0.031220,0.083462,0.049477,0.074432

In [32]:
def recommend(movie):
    
    # index of the input movie given by user
    Index=new_df[new_df['title']==movie].index[0]

    # getting the recommended movie index
    recommend_movie_index=pd.Series(similarity[Index]).sort_values(ascending=False)[1:6].index

    # giving recommendations
    for movies in list(new_df.loc[recommend_movie_index,['movie_id','title']].values):
        print(movies)

In [33]:
recommend("Batman Begins")

[155 'The Dark Knight']
[2661 'Batman']
[268 'Batman']
[49026 'The Dark Knight Rises']
[36597 'Wuthering Heights']


### Pickling the dataset

In [34]:
pickle.dump(similarity,open("similarity.pkl",'wb'))
pickle.dump(new_df,open("movies.pkl",'wb'))